# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [1]:
import requests
import json
import pandas as pd


# Define the URL
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

# Send the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("All good!")
    print("==============\n")
    
    # Convert the response to JSON format (i.e., a nested dictionary)
    data = response.json()
    
    # Extract relevant information (nobelPrizes) from the JSON data
    nobel_prizes = data.get('nobelPrizes', [])
    
    # Convert the list of Nobel Prizes to a pandas DataFrame
    df = pd.json_normalize(nobel_prizes)
    
    # Display the first few rows of the DataFrame
    print(df.head())
    
    # Optionally, save the DataFrame to a CSV file
    df.to_csv('nobel_prizes.csv', index=False)
    
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


All good!

  awardYear dateAwarded  prizeAmount  prizeAmountAdjusted  \
0      1901  1901-11-12       150782             10531894   
1      1901  1901-11-14       150782             10531894   
2      1901  1901-12-10       150782             10531894   
3      1901  1901-11-12       150782             10531894   
4      1901  1901-10-30       150782             10531894   

                                               links  \
0  [{'rel': 'nobelPrize', 'href': 'https://api.no...   
1  [{'rel': 'nobelPrize', 'href': 'https://api.no...   
2  [{'rel': 'nobelPrize', 'href': 'https://api.no...   
3  [{'rel': 'nobelPrize', 'href': 'https://api.no...   
4  [{'rel': 'nobelPrize', 'href': 'https://api.no...   

                                           laureates             category.en  \
0  [{'id': '160', 'knownName': {'en': 'Jacobus H....               Chemistry   
1  [{'id': '569', 'knownName': {'en': 'Sully Prud...              Literature   
2  [{'id': '462', 'knownName': {'en': 'Henry 

# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [2]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']
    # Your code here
    
    # Normalize the JSON data into a DataFrame
    df = pd.json_normalize(prizes_list)
    
    # Extract the required columns and perform transformations
    df['category'] = df['category.en']
    df['dateAwarded'] = pd.to_datetime(df['dateAwarded']).dt.strftime('%Y-%m-%d')
    df['prizeAmount'] = df['prizeAmount']
    df['prizeAmountAdjusted'] = df['prizeAmountAdjusted']
    
    # Calculate the number of laureates and extract their IDs
    df['Number_of_laureates'] = df['laureates'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    df['motivation'] = df['laureates'].apply(lambda x: [laureate['motivation']['en'] for laureate in x if 'motivation' in laureate] if isinstance(x, list) else [])
    df['laureate_ids'] = df['laureates'].apply(lambda x: [laureate['id'] for laureate in x] if isinstance(x, list) else [])
    
    # Select only the desired columns
    df = df[['category', 'dateAwarded', 'prizeAmount', 'prizeAmountAdjusted', 'Number_of_laureates', 'motivation', 'laureate_ids']]
    
    # Display the resulting DataFrame
    print(df.head())
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

                 category dateAwarded  prizeAmount  prizeAmountAdjusted  \
0               Chemistry  1901-11-12       150782             10531894   
1              Literature  1901-11-14       150782             10531894   
2                   Peace  1901-12-10       150782             10531894   
3                 Physics  1901-11-12       150782             10531894   
4  Physiology or Medicine  1901-10-30       150782             10531894   

   Number_of_laureates                                         motivation  \
0                    1  [in recognition of the extraordinary services ...   
1                    1  [in special recognition of his poetic composit...   
2                    2  [for his humanitarian efforts to help wounded ...   
3                    1  [in recognition of the extraordinary services ...   
4                    1  [for his work on serum therapy, especially its...   

  laureate_ids  
0        [160]  
1        [569]  
2   [462, 463]  
3          [1]  
4

# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [3]:
import time
from tqdm import tqdm


ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)


def get_name(laureate):
    if 'fullName' in laureate:
        return laureate['fullName']['en']
    elif 'orgName' in laureate:
        return laureate['orgName']['en']
    return 'Unknown'

def get_gender(laureate):
    if 'gender' in laureate:
        return laureate['gender']
    elif 'orgName' in laureate:
        return 'None'
    return 'Unknown'

def get_birthdate(laureate):
    if 'birth' in laureate and 'date' in laureate['birth']:
        return laureate['birth']['date']
    return 'Unknown'

def get_age(laureate):
    birth_date = get_birthdate(laureate)
    if birth_date != 'Unknown':
        try:
            birth_date = datetime.strptime(birth_date, '%Y-%m-%d')
            # Assuming 'nobelPrizes' is a list, and we take the first award date
            award_date = laureate.get('nobelPrizes', [{}])[0].get('awardYear')
            if award_date:
                award_year = datetime.strptime(award_date, '%Y')
                age = award_year.year - birth_date.year
                return age
        except:
            return 'Unknown'
    return 'Unknown'

def get_city(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('en', 'Unknown')
    return 'Unknown'

def get_country(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('country', {}).get('en', 'Unknown')
    return 'Unknown'

def get_continent(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('continent', {}).get('en', 'Unknown')
    return 'Unknown'

def get_latitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('latitude', 'Unknown')
    return 'Unknown'

def get_longitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('longitude', 'Unknown')
    return 'Unknown'

laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()

        # Your code here

laureates_df = pd.DataFrame(laureates_dict)

laureates_df

  6%|█████▋                                                                                                 | 55/992 [00:15<04:17,  3.64it/s]


KeyboardInterrupt: 

# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [ ]:
# Your code here
# Your code here
country_counts = laureates_df['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'Number_of_awards']

print(country_counts)

,ID
Country,
USA,296
United Kingdom,91
Germany,84
France,63
Russia,30
...,...
Greece,1
Ghana,1
Faroe Islands (Denmark),1
